In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [1]:
from google.colab import files
upload = files.upload()

Saving train.csv to train.csv


In [2]:
import pandas as pd
import numpy as np
import nltk 
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

In [54]:
df = pd.read_csv('train.csv')

In [55]:
df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [56]:
x_df = df['text']

In [57]:
len(x_df)

7613

In [58]:
x = x_df.values

In [59]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [60]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
# at the case of tokenization we tokenize the word and washout english stop word at the same time
def tokenize_word(sentence):
    pure_sentence = re.sub(r'[-,?!@:#;+0-9/$\*]','',sentence)#replace these symbol by space
    pure_sentence1 = re.sub(r'[\'.]','',pure_sentence)
    pure_sentence2 = re.sub(r'[][]','',pure_sentence1)
    pure_sentence3 = re.sub(r'\x89..','',pure_sentence2)
    pure_sentence4 = re.sub(r'[<=>|]','',pure_sentence3)
    pure_sentence5 = re.sub(r'[()]','',pure_sentence4)
    token_word = word_tokenize(pure_sentence5)
    word = list(filter(lambda x:not x.startswith('https' and 'http'),token_word))
    english_stopword = set(stopwords.words('english')) # set of english stopword
    filter_word = [w for w in word if w not in english_stopword] # washout english stop word such as 'and','or','the',etc
    return filter_word

In [61]:
# Now I write a function of replacing words matching regular expression
import re
replacement_patterns = [
 (r'won\'t', 'will not'),
 (r'can\'t', 'cannot'),
 (r'i\'m', 'i am'),
 (r'ain\'t', 'is not'),
 (r'(\w+)\'ll', '\g<1> will'),
 (r'(\w+)n\'t', '\g<1> not'),
 (r'(\w+)\'ve', '\g<1> have'),
 (r'(\w+)\'s', '\g<1> is'),
 (r'(\w+)\'re', '\g<1> are'),
 (r'(\w+)\'d', '\g<1> would')
]
class Regularexpreplacer:
  def __init__(self):
    self.pattern = [(re.compile(match),repl) for (match,repl) in replacement_patterns]
  def replace(self,text):
    k = text
    for (p,repl) in self.pattern:
      k = re.sub(p,repl,k)
    return k

In [62]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [63]:
import re
class RepeatReplacer:
    def __init__(self):
        self.repeat_regexp = r'(\w*)(\w)\2(\w*)'
        self.repl = r'\1\2\3'
    def replace(self, word):
        repl_word = re.sub(self.repeat_regexp ,self.repl, word)
        if repl_word != word:
            return self.replace(repl_word)
        else:
            return repl_word

In [64]:
def get_word(words,c):
  i = c
  while i < (len(words)-c):
    center_word = words[i]
    context_word = words[(i-c):i]+words[(i+1):(i+c+1)]
    yield center_word,context_word
    i+=1

In [65]:
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
target = []
context = []
BOW = []
for j in range(7610):
  tokenize = tokenize_word(Regularexpreplacer().replace(x[j].lower()))
  T = [lemmatizer.lemmatize(token) for token in tokenize]
#  output = [TextBlob(t).correct() for t in T]
  BOW.extend(T)

In [66]:
len(BOW)

69024

In [67]:
# feel free to use this import 
from collections import Counter

## Build a dictionary that maps words to integers
counts = Counter(BOW)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab,1)} 

## use the dict to tokenize each review in reviews_split
## store the tokenized reviews in reviews_ints
reviews_ints = []
for j in range(7613):
  tokenize = tokenize_word(Regularexpreplacer().replace(x[j].lower()))
  T = [lemmatizer.lemmatize(token) for token in tokenize]
  reviews_ints.append([vocab_to_int[word] for word in T])

In [68]:
def pad_features(reviews_ints, seq_length):
    ''' Return features of review_ints, where each review is padded with 0's 
        or truncated to the input seq_length.
    '''
    ## getting the correct rows x cols shape
    features = np.zeros((len(reviews_ints), seq_length), dtype=int)
    
    ## for each review, I grab that review
    for i, row in enumerate(reviews_ints):
      features[i,:len(row)] = np.array(row)[:seq_length]
    
    return features

In [69]:
# Test your implementation!

seq_length = 30

features = pad_features(reviews_ints, seq_length=seq_length)

## test statements - do not change - ##
assert len(features)==len(reviews_ints), "Your features should have as many rows as reviews."
assert len(features[0])==seq_length, "Each feature row should contain seq_length values."

In [70]:
features.shape

(7613, 30)

In [71]:
y = df['target'].values

In [72]:
split_frac = 0.8

## split data into training, validation, and test data (features and labels, x and y)
split_idx = int(len(features)*0.8)
train_x, remaining_x = features[:split_idx], features[split_idx:]
train_y, remaining_y = y[:split_idx], y[split_idx:]

test_idx = int(len(remaining_x)*0.5)
val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]

In [73]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

# dataloaders
batch_size = 10

# make sure to SHUFFLE your data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size,drop_last = True)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size,drop_last = True)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size,drop_last = True)

In [74]:
len(train_loader)

609

In [75]:
# First checking if GPU is available
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

Training on GPU.


In [76]:
import torch.nn as nn

class SentimentRNN(nn.Module):
    """
    The RNN model that will be used to perform Sentiment analysis.
    """

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers,drop_prob=0.5):
        """
        Initialize the model by setting up the layers.
        """
        super(SentimentRNN, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, batch_first=True)
        
        # dropout layer
        self.dropout = nn.Dropout(0.3)
        
        # linear and sigmoid layer
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()

    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size = x.size(0)
        
        # embeddings and lstm_out
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
        
        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        # dropout and fully connected layer
        lstm_out = self.dropout(lstm_out)
        out = self.fc(lstm_out)
        
        # sigmoid function
        sig_out = self.sig(out)
        
        # reshape to be batch_size first
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1] # get last batch of labels
        
        # return last sigmoid output and hidden state
        return sig_out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
          hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                   weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
          hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                   weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden

In [77]:
# Instantiate the model w/ hyperparams
vocab_size = len(vocab_to_int) + 1 # +1 for zero padding + our word tokens
output_size = 1
embedding_dim = 4000
hidden_dim =256
n_layers = 1

net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)

print(net)

SentimentRNN(
  (embedding): Embedding(15488, 4000)
  (lstm): LSTM(4000, 256, batch_first=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (sig): Sigmoid()
)


In [78]:
# loss and optimization functions
lr=0.0001

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

In [79]:
# training params

epochs = 10 # 3-4 is approx where I noticed the validation loss stop decreasing

counter = 0
print_every = 100
clip=5 # gradient clipping

# move model to GPU, if available
if(train_on_gpu):
    net.cuda()

net.train()
h2 = []
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    h = net.init_hidden(batch_size)

    # batch loop
    for inputs, labels in train_loader:
        counter += 1

        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()

        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])
        h2.append(h)
        # zero accumulated gradients
        net.zero_grad()

        # get the output from the model
        output, h = net(inputs, h)

        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()
        

        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for inputs, labels in valid_loader:

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = tuple([each.data for each in val_h])

                if (train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()

                output, val_h = net(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels.float())

                val_losses.append(val_loss.item())

            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

Epoch: 1/10... Step: 100... Loss: 0.548883... Val Loss: 0.693454
Epoch: 1/10... Step: 200... Loss: 0.674191... Val Loss: 0.700313
Epoch: 1/10... Step: 300... Loss: 0.704066... Val Loss: 0.693502
Epoch: 1/10... Step: 400... Loss: 0.691175... Val Loss: 0.697022
Epoch: 1/10... Step: 500... Loss: 0.759227... Val Loss: 0.683965
Epoch: 1/10... Step: 600... Loss: 0.584985... Val Loss: 0.688342
Epoch: 2/10... Step: 700... Loss: 0.749580... Val Loss: 0.683742
Epoch: 2/10... Step: 800... Loss: 0.654761... Val Loss: 0.681693
Epoch: 2/10... Step: 900... Loss: 0.620128... Val Loss: 0.672373
Epoch: 2/10... Step: 1000... Loss: 0.682512... Val Loss: 0.675830
Epoch: 2/10... Step: 1100... Loss: 0.467894... Val Loss: 0.692829
Epoch: 2/10... Step: 1200... Loss: 0.632242... Val Loss: 0.660716
Epoch: 3/10... Step: 1300... Loss: 0.583626... Val Loss: 0.654044
Epoch: 3/10... Step: 1400... Loss: 0.616806... Val Loss: 0.640647
Epoch: 3/10... Step: 1500... Loss: 0.470227... Val Loss: 0.672820
Epoch: 3/10... Step

In [80]:
# Get test data loss and accuracy

test_losses = [] # track loss
num_correct = 0

# init hidden state
#h = net.init_hidden(batch_size)

net.eval()
# iterate over test data
for inputs, labels in test_loader:

    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    h = tuple([each.data for each in h])

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()
    
    # get predicted outputs
    output, h = net(inputs, h)
    
    # calculate loss
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze())  # rounds to the nearest integer
    
    # compare predictions to true label
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)


# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

Test loss: 0.808
Test accuracy: 0.756


In [41]:
from google.colab import files
upload = files.upload()

Saving test.csv to test.csv


In [81]:
df_test = pd.read_csv("test.csv")

In [82]:
df_test

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [83]:
X_test = df_test["text"].values

In [84]:
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
target = []
context = []
BOW1 = []
for j in range(3260):
  tokenize = tokenize_word(Regularexpreplacer().replace(X_test[j].lower()))
  T1 = [lemmatizer.lemmatize(token) for token in tokenize]
  BOW1.extend(T1)

In [85]:
# feel free to use this import 
from collections import Counter

## Build a dictionary that maps words to integers
counts = Counter(BOW1)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab,1)} 

## use the dict to tokenize each review in reviews_split
## store the tokenized reviews in reviews_ints
reviews_ints = []
for j in range(3260):
  tokenize = tokenize_word(Regularexpreplacer().replace(X_test[j].lower()))
  T1 = [lemmatizer.lemmatize(token) for token in tokenize]
  reviews_ints.append([vocab_to_int[word] for word in T1])

In [86]:
seq_length = 30

features1 = pad_features(reviews_ints, seq_length=seq_length)

## test statements - do not change - ##
assert len(features1)==len(reviews_ints), "Your features should have as many rows as reviews."
assert len(features1[0])==seq_length, "Each feature row should contain seq_length values."

In [87]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# create Tensor datasets
data = TensorDataset(torch.from_numpy(features1))

# dataloaders
batch_size = 10

# make sure to SHUFFLE your data
data_loader = DataLoader(data, shuffle=True, batch_size=batch_size)

In [88]:
len(data_loader)

326

In [89]:
# Get test data loss and accuracy

test_losses = [] # track loss
num_correct = 0
y_pred = []
# init hidden state


net.eval()
# iterate over test data
for inputs in data_loader:

    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    h = tuple([each.data for each in h])

    if (train_on_gpu):
      inputs = torch.stack(inputs)
      inputs = inputs.cuda()
      inputs = inputs.squeeze()
    # get predicted outputs
    output1, h = net(inputs, h)
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output1.squeeze()) 
    y_pred.append(pred)

In [91]:
len(y_pred)

326

In [90]:
y_pred

[tensor([1., 0., 1., 1., 1., 0., 0., 1., 0., 0.], device='cuda:0',
        grad_fn=<RoundBackward>),
 tensor([1., 1., 1., 0., 0., 1., 1., 0., 0., 1.], device='cuda:0',
        grad_fn=<RoundBackward>),
 tensor([0., 0., 0., 0., 1., 0., 0., 0., 0., 1.], device='cuda:0',
        grad_fn=<RoundBackward>),
 tensor([0., 0., 0., 1., 0., 1., 1., 1., 0., 0.], device='cuda:0',
        grad_fn=<RoundBackward>),
 tensor([1., 1., 1., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0',
        grad_fn=<RoundBackward>),
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 1., 1.], device='cuda:0',
        grad_fn=<RoundBackward>),
 tensor([0., 1., 0., 1., 0., 0., 0., 0., 0., 1.], device='cuda:0',
        grad_fn=<RoundBackward>),
 tensor([1., 0., 0., 1., 0., 0., 0., 1., 0., 0.], device='cuda:0',
        grad_fn=<RoundBackward>),
 tensor([0., 0., 1., 0., 1., 1., 0., 1., 0., 0.], device='cuda:0',
        grad_fn=<RoundBackward>),
 tensor([1., 0., 0., 1., 1., 1., 1., 0., 1., 1.], device='cuda:0',
        grad_fn=<RoundBa

In [92]:
y_pre=[]
for i in range(len(y_pred)):
  y_pre.append(np.array(y_pred[i].detach().to(torch.device("cpu"))))

In [93]:
np.array(y_pre).flatten()

array([1., 0., 1., ..., 1., 1., 1.], dtype=float32)

In [94]:
len(np.array(y_pre).flatten())

3260

In [95]:
from google.colab import drive
drive.mount('drive')
# to mounting the drive

Mounted at drive


In [96]:
y_hat = pd.DataFrame(np.array(y_pre).flatten())

In [97]:
y_hat

,0
0,1.0
1,0.0
2,1.0
3,1.0
4,1.0
...,...
3255,0.0
3256,1.0
3257,1.0
3258,1.0


In [98]:
kframe = [df_test["text"],y_hat]

In [100]:
ki=pd.concat(kframe,axis=1)

In [101]:
ki=pd.DataFrame(ki)

In [105]:
ki

,text,0
0,Just happened a terrible car crash,1.0
1,"Heard about #earthquake is different cities, s...",0.0
2,"there is a forest fire at spot pond, geese are...",1.0
3,Apocalypse lighting. #Spokane #wildfires,1.0
4,Typhoon Soudelor kills 28 in China and Taiwan,1.0
...,...,...
3258,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...,1.0
3259,Storm in RI worse than last hurricane. My city...,1.0
3260,Green Line derailment in Chicago http://t.co/U...,NaN
3261,MEG issues Hazardous Weather Outlook (HWO) htt...,NaN


In [107]:
ki.to_csv('disaster_prediction.csv')
# to save the result in my drive of google colab